### Analyzing all-dates descriptives

In [1]:
%matplotlib inline
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import datetime as dt
import time

from importlib import reload
from Utils import TransantiagoConstants
from RunSilentlyDailyEtapasBuilder import RunSilentlyDailyEtapasBuilderClass

In [2]:
dates = TransantiagoConstants.currentSSHDates

* Not necessary again

In [3]:
#tic = time.clock()

#for date in dates:
#    etapas_builder = RunSilentlyDailyEtapasBuilderClass(date)
#    etapas_builder.runSimplifyEtapas()
#    del etapas_builder

#toc = time.clock()
#toc-tic

* Continuing with analyses.
* <strike>Consider to refactor this block in order to get number of observations.</strike>
* <strike>Consider to refactor this to get number of trx per database.</strike>
* TODO: Consider to refactor this to get number of trx per database per operator -> this could be computationally expensive and not-so-easy, since it would be necessary to have some dictionary of operators and routes, and search between etapas and the dictionary.
* <strike>Consider to refactor this to get number of already processed databases.</strike>
* Methodology is not robust to potential databases particularities. <strong>Be aware of 2017-04-10 and 2017-04-17: outliers</strong>. Identification of issues with these particular databases should be done from outside
* Identification of special dates should be done from outside.
* Complete process (for all available dates) took about 35 minutes.

* Filtering to refactor

In [4]:
#dates = dates[:50]

* End filtering

In [5]:
#Raw number of trx per day: before cleaning
total_raw_trx = []

#Number of trx per day by turnstile condition: after cleaning but before filtering.
total_trx = []
total_turns_trx = []
total_no_turns_trx = []

#Descriptives: after cleaning and after filtering
turnstile_obs = []
turnstile_means = []
turnstile_medians = []
turnstile_stds = []
turnstile_maxs = []

no_turnstile_obs = []
no_turnstile_means = []
no_turnstile_medians = []
no_turnstile_stds = []
no_turnstile_maxs = []

quantile = 0.97
processed = 0

tic = time.clock()

for date in dates:
    etapas_builder = RunSilentlyDailyEtapasBuilderClass(date)
    [etapas_df, processed_sorted_df, filtered_df, filtered_turnstile_df, filtered_no_turnstile_df]=etapas_builder.runCompleteProcess()
    
    #Filling trx info:
    total_raw_trx.append(etapas_df.count()[0])
    total_trx.append(processed_sorted_df.count()[0])
    total_turns_trx.append(processed_sorted_df[processed_sorted_df.si_2017_torniquete==True].count()[0])
    total_no_turns_trx.append(processed_sorted_df[processed_sorted_df.si_torniquete==False].count()[0])
        
    #Filtering and following the original methodology:
    turnstile_q = filtered_turnstile_df.diferencia_tiempo.quantile(quantile)
    no_turnstile_q = filtered_no_turnstile_df.diferencia_tiempo.quantile(quantile)    
    
    filtered_turnstile_q_df = filtered_turnstile_df[filtered_turnstile_df.diferencia_tiempo<=turnstile_q]
    filtered_no_turnstile_q_df = filtered_no_turnstile_df[filtered_no_turnstile_df.diferencia_tiempo<=no_turnstile_q]
    
    turnstile_obs.append(filtered_turnstile_q_df.count()[0])
    turnstile_means.append(filtered_turnstile_q_df['diferencia_tiempo_secs'].mean())
    turnstile_stds.append(filtered_turnstile_q_df['diferencia_tiempo_secs'].std())
    turnstile_medians.append(filtered_turnstile_q_df['diferencia_tiempo_secs'].quantile(0.50))
    turnstile_maxs.append(filtered_turnstile_q_df['diferencia_tiempo_secs'].max())
    
    no_turnstile_obs.append(filtered_no_turnstile_q_df.count()[0])
    no_turnstile_means.append(filtered_no_turnstile_q_df['diferencia_tiempo_secs'].mean())
    no_turnstile_stds.append(filtered_no_turnstile_q_df['diferencia_tiempo_secs'].std())    
    no_turnstile_medians.append(filtered_no_turnstile_q_df['diferencia_tiempo_secs'].quantile(0.50))
    no_turnstile_maxs.append(filtered_no_turnstile_q_df['diferencia_tiempo_secs'].max())
    
    processed = processed + 1
    print('Number of already processed ddbb is ' + str(processed))
    del etapas_builder
    
toc = time.clock()  
toc-tic

Not found in turnstile database: 1614656
Number of already processed ddbb is 1
Not found in turnstile database: 1641812
Number of already processed ddbb is 2
Not found in turnstile database: 1689770
Number of already processed ddbb is 3
Not found in turnstile database: 1033091
Number of already processed ddbb is 4
Not found in turnstile database: 656097
Number of already processed ddbb is 5
Not found in turnstile database: 1776322
Number of already processed ddbb is 6
Not found in turnstile database: 1795630
Number of already processed ddbb is 7
Not found in turnstile database: 1079588
Number of already processed ddbb is 8
Not found in turnstile database: 622999
Number of already processed ddbb is 9
Not found in turnstile database: 1824234
Number of already processed ddbb is 10
Not found in turnstile database: 1832615
Number of already processed ddbb is 11
Not found in turnstile database: 1842842
Number of already processed ddbb is 12
Not found in turnstile database: 1821215
Number of 

Not found in turnstile database: 705670
Number of already processed ddbb is 104
Not found in turnstile database: 1876347
Number of already processed ddbb is 105
Not found in turnstile database: 1867643
Number of already processed ddbb is 106
Not found in turnstile database: 1707189
Number of already processed ddbb is 107
Not found in turnstile database: 1081146
Number of already processed ddbb is 108
Not found in turnstile database: 584930
Number of already processed ddbb is 109
Not found in turnstile database: 1538951
Number of already processed ddbb is 110
Not found in turnstile database: 716738
Number of already processed ddbb is 111
Not found in turnstile database: 1873694
Number of already processed ddbb is 112
Not found in turnstile database: 1888514
Number of already processed ddbb is 113
Not found in turnstile database: 1106433
Number of already processed ddbb is 114
Not found in turnstile database: 665660
Number of already processed ddbb is 115
Not found in turnstile database:

2478.8531453899445

* Summarizing

In [6]:
dates = [dt.datetime.strptime(x, "%Y-%m-%d").date() for x in dates]

In [7]:
summary_descriptives = pd.DataFrame(
    {'date': dates,
     'total_raw_trx': total_raw_trx,
     'total_trx': total_trx,
     'total_turns_trx': total_turns_trx,
     'total_no_turns_trx': total_no_turns_trx,
     'turnstile_obs':turnstile_obs,
     'turnstile_means': turnstile_means,
     'turnstile_stds': turnstile_stds,
     'turnstile_medians': turnstile_medians,
     'turnstile_maxs': turnstile_maxs,
     'no_turnstile_obs': no_turnstile_obs,
     'no_turnstile_means': no_turnstile_means,
     'no_turnstile_stds': no_turnstile_stds,
     'no_turnstile_medians': no_turnstile_medians,
     'no_turnstile_maxs': no_turnstile_maxs})

In [8]:
cols = ['date','total_raw_trx','total_trx','total_turns_trx','total_no_turns_trx',
        'turnstile_obs','turnstile_means','turnstile_stds','turnstile_medians','turnstile_maxs',
        'no_turnstile_obs','no_turnstile_means','no_turnstile_stds', 'no_turnstile_medians', 'no_turnstile_maxs']
summary_descriptives = summary_descriptives[cols]

* Checking

In [9]:
summary_descriptives

,date,total_raw_trx,total_trx,total_turns_trx,total_no_turns_trx,turnstile_obs,turnstile_means,turnstile_stds,turnstile_medians,turnstile_maxs,no_turnstile_obs,no_turnstile_means,no_turnstile_stds,no_turnstile_medians,no_turnstile_maxs
0,2017-03-01,2329229,2210399,48762,1730490,29579,6.473275,5.615842,5.0,43.0,1063089,3.932738,5.192272,3.0,40.0
1,2017-03-02,2369991,2332086,54086,1824820,32814,5.832876,3.873914,5.0,25.0,1103956,3.433931,3.537053,3.0,25.0
2,2017-03-03,2448178,2410476,53944,1884238,32917,5.847313,4.011833,5.0,26.0,1147970,3.405038,3.432229,3.0,24.0
3,2017-03-04,1533875,1508121,36319,1158202,20864,6.245830,4.247982,5.0,26.0,663497,3.489298,3.432341,3.0,24.0
4,2017-03-05,982153,964519,22593,732016,12004,6.427358,4.434549,5.0,27.0,391231,3.524174,3.413945,3.0,24.0
5,2017-03-07,2567943,2529902,56546,1972070,34850,5.738020,3.932266,5.0,26.0,1216096,3.439889,3.544536,3.0,25.0
6,2017-03-09,2574321,2535246,55117,1987815,33903,5.626995,3.722527,5.0,25.0,1221041,3.399090,3.441014,2.0,24.0
7,2017-03-11,1569253,1544112,34153,1196515,19046,6.064265,4.081304,5.0,25.0,676705,3.423354,3.402772,3.0,24.0
8,2017-03-12,934524,916257,19436,700690,9519,6.271562,4.249260,5.0,25.0,362783,3.539846,3.496062,3.0,25.0
9,2017-03-13,2619641,2576017,54775,2014285,33388,5.685066,3.776007,5.0,25.0,1258365,3.426656,3.452691,3.0,24.0


* Writting the ddbb to a file

In [16]:
#summary_descriptives.to_csv('C:/Users/Tesista/Desktop/Evasion/01_analisis/03_datos/05_SUMMARY/DescriptivesSummary.csv', sep=';', encoding = 'latin-1')

### Closed